In [1]:
import numpy
import cupy

import scipy
import scipy.sparse.linalg
from scipy.sparse.linalg import splu
import cupyx

In [2]:
mempool = cupy.get_default_memory_pool()

with cupy.cuda.Device(0):
    mempool.set_limit(size=3*1024**3)  # 2 GiB

# Dense matrices performance

High-D, numpy, no correlation

In [3]:
large_dim = 100000
small_dim = 100

tall = False

if tall:
    cols = small_dim
    rows = large_dim
else:
    cols = large_dim
    rows = small_dim


numpy.random.seed(42)
G = scipy.sparse.csr_matrix(
    scipy.sparse.random(rows, cols, density=0.01, random_state=42).T
)
G_csc = scipy.sparse.csc_matrix(G)

print(G.shape)

m = numpy.ones((G.shape[1], 1))
d0 = 0.2 * numpy.ones((G.shape[0], 1))

# pre-transpose
Gt = G.T
Gt_csr = scipy.sparse.csr_matrix(Gt)
d0t = d0.T

covariance = 1.2 * numpy.ones_like(d0)
sigma = covariance ** 0.5

print(f"{0.5 * numpy.linalg.norm((G @ m - d0) / sigma) ** 2:.2f}")
print(f"{0.5 * numpy.linalg.norm((G_csc @ m - d0) / sigma) ** 2:.2f}")
print(f"{0.5 * ((m.T @ Gt - d0t) @ ((G @ m - d0) / covariance)).item():.2f}")
print(f"{0.5 * ((m.T @ Gt_csr - d0t) @ ((G @ m - d0) / covariance)).item():.2f}")
print(f"{0.5 * ((m.T @ Gt - d0t) @ ((G_csc @ m - d0) / covariance)).item():.2f}")
print(f"{0.5 * ((m.T @ Gt_csr - d0t) @ ((G_csc @ m - d0) / covariance)).item():.2f}")

%timeit -n 50 0.5 * numpy.linalg.norm((G @ m - d0) / sigma) ** 2 # Fastest for wide
%timeit -n 50 0.5 * numpy.linalg.norm((G_csc @ m - d0) / sigma) ** 2 # Fastest for wide
%timeit -n 50 0.5 * ((m.T @ Gt - d0t) @ ((G @ m - d0) / covariance)).item()
%timeit -n 50 0.5 * ((m.T @ Gt_csr - d0t) @ ((G @ m - d0) / covariance)).item()
%timeit -n 50 0.5 * ((m.T @ Gt - d0t) @ ((G_csc @ m - d0) / covariance)).item()
%timeit -n 50 0.5 * ((m.T @ Gt_csr - d0t) @ ((G_csc @ m - d0) / covariance)).item()

(100000, 100)
17482.52
17482.52
17482.52
17482.52
17482.52
17482.52
4.19 ms ± 552 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)
2.85 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)
6.55 ms ± 257 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)
5.37 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)
6.31 ms ± 554 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


KeyboardInterrupt: 

High-D, numpy, correlation

In [ ]:
covariance = (
    1.2 * scipy.sparse.eye(d0.size)
    - 0.6 * scipy.sparse.eye(d0.size, k=-1)
    - 0.6 * scipy.sparse.eye(d0.size, k=1)
)

%time factorized_covariance = scipy.sparse.linalg.factorized( covariance )
%time spcov = splu(covariance)

print(f"{0.5 * ((G @ m - d0).T @ scipy.sparse.linalg.spsolve(covariance, G @ m - d0)).item():.2f}")
print(f"{0.5 * ((G @ m - d0).T @ spcov.solve((G @ m - d0))).item():.2f}")
print(f"{0.5 * ((G @ m - d0).T @ factorized_covariance((G @ m - d0))).item()}")
print(f"{0.5 * ((m.T @ Gt - d0t) @ factorized_covariance((G @ m - d0))).item():.2f}")
print(f"{0.5 * ((m.T @ Gt - d0t) @ factorized_covariance((G_csc @ m - d0))).item():.2f}")
print(f"{0.5 * ((m.T @ Gt_csr - d0t) @ factorized_covariance((G @ m - d0))).item():.2f}")
print(f"{0.5 * ((m.T @ Gt_csr - d0t) @ factorized_covariance((G_csc @ m - d0))).item():.2f}")

# %timeit -n 50 0.5 * ((G @ m - d0).T @ scipy.sparse.linalg.spsolve(covariance, G @ m - d0)).item()
%timeit -n 50 0.5 * ((G @ m - d0).T @ factorized_covariance((G @ m - d0))).item() # Fastest if tall
%timeit -n 50 0.5 * ((m.T @ Gt - d0t) @ factorized_covariance((G @ m - d0))).item() 
%timeit -n 50 0.5 * ((m.T @ Gt - d0t) @ factorized_covariance((G_csc @ m - d0))).item()
%timeit -n 50 0.5 * ((m.T @ Gt_csr - d0t) @ factorized_covariance((G @ m - d0))).item()
%timeit -n 50 0.5 * ((m.T @ Gt_csr - d0t) @ factorized_covariance((G_csc @ m - d0))).item() # Fastest if wide
# %timeit -n 50 0.5 * ((G @ m - d0).T @ spcov.solve((G @ m - d0))).item()


High-D, cupy, no correlation

In [ ]:
G_gpu = cupyx.scipy.sparse.csr_matrix(G)
G_csc_gpu = cupyx.scipy.sparse.csc_matrix(G_gpu)

d0_gpu = cupy.array(d0)
Gt_gpu = G_gpu.T



covariance_gpu = 1.2 * cupy.ones_like(d0_gpu)
sigma_gpu = covariance_gpu ** 0.5

print(
    f"{(0.5 * cupy.linalg.norm((G_csc_gpu.dot(cupy.array(m)) - d0_gpu) / sigma_gpu) ** 2).item():.2f}"
)
print(
    f"{(0.5 * cupy.linalg.norm((G_gpu.dot(cupy.array(m)) - d0_gpu) / sigma_gpu) ** 2).item():.2f}"
)
print(
    f"{0.5* ((G_gpu.dot(cupy.array(m)) - d0_gpu).T @ ((G_gpu.dot(cupy.array(m)) - d0_gpu) / covariance_gpu)).item():.2f}"
)

%timeit -n 50 0.5 * (cupy.linalg.norm((G_gpu.dot(cupy.array(m)) - d0_gpu) / sigma_gpu) ** 2).item() # Fastest always
# %timeit -n 50 0.5 * (cupy.linalg.norm((G_csc_gpu.dot(cupy.array(m)) - d0_gpu) / sigma_gpu) ** 2).item()
# %timeit -n 50 0.5* ((G_gpu.dot(cupy.array(m)) - d0_gpu).T @ ((G_gpu.dot(cupy.array(m)) - d0_gpu) / covariance_gpu)).item()

High-D, cupy, correlation

In [ ]:
# covariance_gpu = (
#     1.2 * cupyx.scipy.sparse.eye(d0.size)
#     - 0.6 * cupyx.scipy.sparse.eye(d0.size, k=-1)
#     - 0.6 * cupyx.scipy.sparse.eye(d0.size, k=1)
# )

# import cupyx.scipy.linalg

# spcov = splu(covariance)

# L_gpu = cupyx.scipy.sparse.csr_matrix(spcov.L)
# U_gpu = cupyx.scipy.sparse.csr_matrix(spcov.U)
# C_t = spcov.perm_c
# R_t = spcov.perm_r
# A_t = scipy.sparse.eye(R.shape[0])
# R_gpu = cupyx.scipy.sparse.csr_matrix(A[R_t])
# C_gpu = cupyx.scipy.sparse.csr_matrix(A[C_t])


# print(f"{0.5 *((G_gpu.dot(cupy.array(m)) - d0_gpu).T @ C_gpu.dot(cupyx.scipy.sparse.linalg.lsqr(U, (cupyx.scipy.sparse.linalg.lsqr(L_gpu, R_gpu.dot(G_gpu.dot(cupy.array(m)) - d0_gpu)[:,0]))[0])[0][:,None])).item():.2f}")
# print(f"{0.5 * ((G_gpu.dot(cupy.array(m)) - d0_gpu).T @ cupyx.scipy.sparse.linalg.lsqr(covariance_gpu, (G_gpu.dot(cupy.array(m)) - d0_gpu)[:,0])[0][:,None]).item():.2f}")

# print("TOO SLOW")

# %timeit -n 50 0.5 *((G_gpu.dot(cupy.array(m)) - d0_gpu).T @ C_gpu.dot(cupyx.scipy.sparse.linalg.lsqr(U, (cupyx.scipy.sparse.linalg.lsqr(L_gpu, R_gpu.dot(G_gpu.dot(cupy.array(m)) - d0_gpu)[:,0]))[0])[0][:,None])).item()
# %timeit -n 50 0.5 * ((G_gpu.dot(cupy.array(m)) - d0_gpu).T @ cupyx.scipy.sparse.linalg.lsqr(covariance_gpu, (G_gpu.dot(cupy.array(m)) - d0_gpu)[:,0])[0][:,None]).item()

Winner: Depends on what you want.

`i7-8850H vs Quadro P2000`

## Gradients

In [ ]:
large_dim = 1000000
small_dim = 100

# GPU outperforms: ~ 1000000 nnz elements

tall = True

if tall:
    cols = small_dim
    rows = large_dim
else:
    cols = large_dim
    rows = small_dim


numpy.random.seed(42)
G = scipy.sparse.csr_matrix(
    scipy.sparse.random(rows, cols, density=0.01, random_state=42).T
)

G_csc = scipy.sparse.csc_matrix(G)

print(G.shape)

m = numpy.ones((G.shape[1], 1))
d0 = 0.2 * numpy.ones((G.shape[0], 1))

# pre-transpose
Gt = scipy.sparse.csc_matrix(G.T)
Gt_csr = scipy.sparse.csr_matrix(Gt)
d0t = d0.T

covariance = 1.2 * numpy.ones_like(d0)
sigma = covariance ** 0.5

print(f"{((G.T @ ((G @ m - d0) / covariance)).T @ m).item():.2f}")
print(f"{((Gt @ ((G @ m - d0) / covariance)).T @ m).item():.2f}")
print(f"{((Gt_csr @ ((G @ m - d0) / covariance)).T @ m).item():.2f}")

print(
    "Equivalence:",
    numpy.allclose(
        (G.T @ ((G @ m - d0) / covariance)), (Gt @ ((G @ m - d0) / covariance))
    )
    and numpy.allclose(
        (G.T @ ((G @ m - d0) / covariance)), (Gt_csr @ ((G @ m - d0) / covariance))
    ),
)

%timeit -n 50 Gt @ ((G_csc @ m - d0) / covariance) 
%timeit -n 50 Gt_csr @ ((G @ m - d0) / covariance) 
%timeit -n 50 Gt @ ((G @ m - d0) / covariance)  # faster if tall
%timeit -n 50 Gt_csr @ ((G_csc @ m - d0) / covariance) # faster if wide

In [ ]:
covariance = (
    1.2 * scipy.sparse.eye(d0.size)
    - 0.6 * scipy.sparse.eye(d0.size, k=-1)
    - 0.6 * scipy.sparse.eye(d0.size, k=1)
)
import scipy.sparse.linalg

%time factorized_covariance = scipy.sparse.linalg.factorized( covariance )


print(f"{((G.T @ factorized_covariance((G @ m - d0))).T @ m).item():.2f}")

print(
    "Equivalence:",
    numpy.allclose(
        G.T @ factorized_covariance((G @ m - d0)),
        Gt_csr @ factorized_covariance(G_csc @ m - d0),
    )
    and numpy.allclose(
        Gt @ factorized_covariance(G @ m - d0),
        Gt_csr @ factorized_covariance(G_csc @ m - d0),
    )
    and numpy.allclose(
        Gt @ factorized_covariance(G_csc @ m - d0),
        Gt_csr @ factorized_covariance(G_csc @ m - d0),
    )
    and numpy.allclose(
        Gt_csr @ factorized_covariance(G @ m - d0),
        Gt_csr @ factorized_covariance(G_csc @ m - d0),
    )
)

%timeit -n 50 Gt @ factorized_covariance(G_csc @ m - d0)
%timeit -n 50 Gt_csr @ factorized_covariance(G @ m - d0)
%timeit -n 50 Gt @ factorized_covariance(G @ m - d0) # Fastest if tall
%timeit -n 50 Gt_csr @ factorized_covariance(G_csc @ m - d0) # Fastest if wide

In [ ]:
G_gpu = cupyx.scipy.sparse.csr_matrix(G, dtype="float32")
G_csc_gpu = cupyx.scipy.sparse.csc_matrix(G_gpu)
Gt_gpu = G_gpu.T
Gt_csr_gpu = cupyx.scipy.sparse.csr_matrix(Gt_gpu, dtype="float32")
d0_gpu = cupy.array(d0, dtype="float32")
covariance_gpu = 1.2 * cupy.ones_like(d0_gpu, dtype="float32")
sigma_gpu = covariance_gpu ** 0.5

print(
    f"{((Gt_gpu.dot(((G_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)).T @ cupy.asarray(m)).item():.2f}"
)

print(
    "Equivalence:",
    numpy.allclose(
        (Gt_gpu.dot(((G_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)),
        (Gt_csr_gpu.dot(((G_csc_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)),
    )
    and numpy.allclose(
        (Gt_gpu.dot(((G_csc_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)),
        (Gt_csr_gpu.dot(((G_csc_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)),
    )
    and numpy.allclose(
        (Gt_csr_gpu.dot(((G_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)),
        (Gt_csr_gpu.dot(((G_csc_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)),
    ),
)

print(G.shape)

# %timeit -n 50 (Gt_csr_gpu.dot(((G_csc_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)).get()
# %timeit -n 50 (Gt_gpu.dot(((G_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)).get()
# %timeit -n 50 (Gt_gpu.dot(((G_csc_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)).get()
%timeit -n 50 (Gt_csr_gpu.dot(((G_gpu.dot(cupy.array(m)) - d0_gpu)) / covariance_gpu)).get() # Fastest always